In [ ]:
### 必要なライブラリのインポート
import os
import sys
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
from mlflow.models.signature import infer_signature
import mlflow.sklearn

from ipywidgets import interact

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


In [ ]:
###この関数 eval_metrics() は、モデルの予測精度を評価するための3つの指標を計算する関数です。
def eval_metrics(actual, pred):
    ## Root Mean Squared Error（平方根平均二乗誤差）: 小さいほど予測が実際に近い（誤差が小さい）
    rmse = np.sqrt(mean_squared_error(actual,pred))
    ## Mean Absolute Error（平均絶対誤差）:	誤差の平均。これも小さいほど良い
    mae = mean_absolute_error(actual, pred)
    ## R-squared（決定係数）: 1に近いほど「よく説明できている」
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
if __name__=="__main__":
    ## データの読み込み
    csv_url=("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv")

    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e :
        logger.exception("Unable to download the data")
    ## データの分割
    train, test = train_test_split(data)

    train_x = train.drop(["quality"], axis = 1)
    test_x = test.drop(["quality"], axis = 1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    ## モデルのパラメータ設定
    alpha =  0.5
    l1_ratio =  0.5

    with mlflow.start_run():
        ## モデルの訓練と予測
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        
        ## 評価指標の計算と表示
        (rmse, mae, r2)  = eval_metrics(test_y, predicted_qualities)
        print("Elasticnet model (alpha={:f}, l1_ratio={:f})".format(alpha, l1_ratio))
        print(" RMSE: %s" % rmse)
        print(" MAE: %s" % mae)
        print(" R2: %s" % r2)

        ## MLflow に記録（ログ）
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        ## MLflow の記録先を設定
        remote_server_uri = "http://ec2-3-115-116-130.ap-northeast-1.compute.amazonaws.com:5000/"
        mlflow.set_tracking_uri(remote_server_uri)
        
        ## モデルの保存
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        ## MLflow tracking URI がローカルファイルじゃない場合（たとえば AWS EC2 の HTTP URI のとき）：→
        # モデルを「ElasticnetWineModel」としてレジストリに登録（バージョン管理される）
        if tracking_url_type_store!="file":
            mlflow.sklearn.log_model(
                lr,"model", registered_model_name="ElsaticnetWineModel"
            )
        ## ローカルファイルの場合は、普通にモデルだけ保存（レジストリには登録しない）
        else:
            mlflow.sklearn.log_model(lr, "model")

2025/08/06 15:36:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Elasticnet model (alpha=0.500000, l1_ratio=0.500000)
 RMSE: 0.756652810029142
 MAE: 0.6057857247169067
 R2: 0.11877946351646929
